In [6]:
import torch
from models.yolo import Model
import yaml

In [7]:
cfg_path = "models/yolov5_custom.yaml"
hyp_path = "data/hyps/hyp.scratch-low.yaml"
nc = 1

with open(hyp_path, errors="ignore") as f:
            hyp = yaml.safe_load(f)  # load hyps dict


with open(cfg_path, errors="ignore") as f:
            cfg = yaml.safe_load(f)  # load cfg dict

model = Model(cfg, ch=3, nc=nc, anchors=hyp.get("anchors"))


                 from  n    params  module                                  arguments                     


  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1    192545  models.freqspa.MixedBlock               [64, 64]                      
  4                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  5                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  6                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  7                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  8                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  9                -1  1   1182720  m

In [1]:
import torch
from torch import nn

In [19]:
class Network(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv = nn.Conv2d(in_c, out_c, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(out_c)
        self.act = nn.SiLU()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.conv2 = nn.Conv2d(out_c, out_c, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.act(self.bn(self.conv(x)))
        x = self.pool(x)
        x = self.conv2(x)
        return x

In [3]:
grads = []
acts = []

def backward_hook(module, grad_input, grad_output):
    grads.append(grad_output[0].detach())
    return None

def forward_hook(module, input, output):
    acts.append(output.detach())
    return None

In [22]:
net = Network(3, 64)
net.act.register_forward_hook(forward_hook)
net.act.register_full_backward_hook(backward_hook)
input = torch.randn(1, 3, 128, 128)


out = net(input)

for i in range(64):
    net.zero_grad()
    out[0, i, 0, 0].backward(retain_graph=True)
    print(acts[0].shape, grads[0].shape)

torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size([1, 64, 128, 128])
torch.Size([1, 64, 128, 128]) torch.Size

In [17]:
acts[0].shape

torch.Size([1, 64, 128, 128])

In [18]:
grads[0].shape

torch.Size([1, 64, 128, 128])